## Rough Heston model

In the rough Heston model, the stock price satisfies 
$$\mathrm{d}S_t=rS_t\mathrm{d} t + S_t\sqrt{V_s}\mathrm{d} W_t
$$
where 
$$V_t  = V_0 +   \frac{1}{\Gamma(1-\alpha)}\int_0^t  (t-s)^{-\alpha} \kappa(\theta - V_s) \mathrm{d} s +   \frac{1}{\Gamma(1-\alpha)}\int_0^t  (t-s)^{-\alpha}\nu\sqrt{V_s} \mathrm{d} B_s,
$$
and the two Brownian motions are correlated as
$\mathrm{d} \langle W, B\rangle_t = \rho\, \mathrm{d} t$,
for $\alpha \in \left(\frac{1}{2},1\right)$, with $S_0,V_0>0$, and $W, B$ are two correlated Brownian motions.

($\kappa\epsilon=\nu$)

When $\alpha =0 $, it reduces to the classical Heston model of Heston (1993).

It is hard to simulate the paths of the rough Heston model, 
as (2) contains an integral process and the kernel $(t − s)^{−\alpha}$ with $\alpha \in (0, 1/2)$ is singular at point s = t. To simulate a path at a time level, one needs to keep all the past information. So the simulation is extremely time-consuming and the complexity of the Euler-Maruyama algorithm for simulating one path is proportional to O($N^{2}$) which is too large. 

Moreover, since the integrand is singular at s = t, the discretization of the integral process needs special treatment to attain good accuracy. 

### ALGORITHM 1 Modified Euler-Maruyama Algorithm $\nu$

In [120]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import bisect
from scipy.special import gamma,roots_jacobi ,roots_legendre
from scipy.integrate import quad, trapz, simps
import pandas as pd
from functools import reduce


In [166]:
scale = 1
n_quad = 10
x, w = roots_jacobi(n_quad, alpha=1, beta=1.5)

x *= scale
w /= w.sum()
x, w

(array([-0.92758001, -0.79101274, -0.59842537, -0.36348091, -0.10287083,
         0.16488663,  0.42076434,  0.64657859,  0.82627988,  0.94708264]),
 array([0.00305011, 0.02204033, 0.06805619, 0.13428174, 0.19377731,
        0.21515904, 0.18491321, 0.11850472, 0.05042276, 0.00979459]))

In [167]:
scale = 1
n_quad = 10
x, w = roots_legendre(n_quad)

x *= scale
w /= w.sum()
x, w

(array([-0.97390653, -0.86506337, -0.67940957, -0.43339539, -0.14887434,
         0.14887434,  0.43339539,  0.67940957,  0.86506337,  0.97390653]),
 array([0.03333567, 0.07472567, 0.10954318, 0.13463336, 0.14776211,
        0.14776211, 0.13463336, 0.10954318, 0.07472567, 0.03333567]))

In [161]:
class RoughHeston:
    '''

    '''
    def __init__(self, texp, V0=0, alpha=0.0, kappa=0.3, theta=0.5, nu=0.01, time_steps=1_000):
        self.texp = texp
        self.V0 = V0
        self.alpha = alpha
        self.kappa = kappa
        self.theta = theta
        self.nu = nu               # need to be small to garantee V>0
        self.time_steps = time_steps
               
    def volMEM(self, texp, V0=0, alpha=0.0, kappa=0.3, theta=0.5, nu=0.01, time_steps=1_000):
        '''
        
        '''
        z = np.random.normal(size=(time_steps+1)) 
        V_tk = V0 * np.ones(time_steps+1) 
        tk=np.linspace(0, texp, num=time_steps+1)  

        for i in range(self.time_steps+1):
            fV=0
            gV=0
            for j in range(i):    # k=j+1
                fV = fV + kappa * (theta-V_tk[j]) * ((tk[i]-tk[j])**(1-alpha)-(tk[i]-tk[j+1])**(1-alpha))
            for j in range(i):
                gV = gV+ nu * np.sqrt(V_tk[j]) * np.sqrt(((tk[i]-tk[j])**(1-2*alpha)-(tk[i]-tk[j+1])**(1-2*alpha))/(1-2*alpha))*z[j+1]
            V_tk[i] = np.max((V_tk[0] + 1/gamma(2.- alpha) * fV + 1/gamma(1.- alpha) * gV),0)      
        return V_tk

        #I = simps(sigma_tk * sigma_tk, dx=texp/self.time_steps) / (self.sigma**2)  # integrate by using Simpson's rule
        
    def volFast(self, texp, n_exp=10, V0=0, alpha=0.0, kappa=0.3, theta=0.5, nu=0.01, time_steps=1_000):    
        
        '''
        '''
        tau = texp/time_steps
        xi = 0.0001 
        M = 0.5*np.log(texp)//1  #Assume O(N)=0.5(N)
        #Gauss quadrature parameters
        scale = 1
        n_quad = 2* n_exp
        #Gauss-Jacobi quadrature on the interval [0, 2^(−M)]
        x, w = roots_jacobi(n_quad, alpha=1, beta=1.5)
        x *= scale
        w /= w.sum()
        tar=np.intersect1d(np.argwhere(x>=0),np.argwhere(x<=2**(-M)))
        s0 = x[tar]
        w0 = w[tar]
        #nodes and weights for the ns-point Gauss-Legendre quadrature on the small interval [2^j,2^(j+1)], j =−M, ... , −1
        x, w = roots_legendre(n_quad)

        x *= scale
        w /= w.sum()
        tar=np.intersect1d(np.argwhere(x>2**(-M)),np.argwhere(x<=1))
        sjns = x[tar]
        wjns = w[tar]
        #nodes and weights for the nl-point Gauss-Legendre quadrature on the small interval [2^j,2^(j+1)], j = 0, ... , N
        tar=np.intersect1d(np.argwhere(x>1),np.argwhere(x<=2**(time_steps+1)))
        sjnl = x[tar] 
        wjnl = w[tar]
    
        #xl,wl
        xl = reduce(np.union1d,[s0,sjns,sjnl])
        wl = reduce(np.union1d,[w0,sjns**(alpha-1)*wjns,sjnl**(alpha-1)*wjnl])/gamma(alpha)
        N_exp = xl.size
        
       # N = (np.log(np.log(1.0/xi)) + np.log(time_steps / texp))//1           *0.5
       # N_exp = np.log(1.0/xi) * ( np.log(np.log(1.0/xi)) + np.log(time_steps)) \
        #+ np.log(time_steps / texp)*( np.log(np.log(1.0/xi)) * np.log(time_steps / texp))   *0.5

        z = np.random.normal(size=(time_steps+1)) 
        V_tk = V0 * np.ones(time_steps+1) 
        H_tn =np.zeros([time_steps+1,N_exp])
        J_tn =np.zeros([time_steps+1,N_exp])
        tk=np.linspace(0, texp, num=time_steps+1)  

        for i in range(1,self.time_steps+1):
            #HJ matrix update by row
            for j in range(N_exp):  #perhaps no need to j loop
                if i>1:
                    H_tn[i-1,j] = kappa * (theta-V_tk[i-2])/xl[j]*(1-np.exp(-xl[j]*tau)) + np.exp(-xl[j]*tau)*H_tn[i-2,j]
                
            for j in range(N_exp):
                if i>1:
                    J_tn[i-1,j] = np.exp(-xl[j]*tau)* nu * np.sqrt(V_tk[i-2]*tau)*z[i-1] + np.exp(-xl[j]*tau)*J_tn[i-2,j]
                    
            
            V_tk[i] = np.max((V_tk[0] + tau**(1.-alpha)/gamma(2.- alpha) * kappa * (theta-V_tk[i-1])\
                      + 1/gamma(1.- alpha) * wl*np.exp(-xl*tau)@H_tn[i-1,:]      \
                      + tau**(0.5 - alpha)/gamma(1.- alpha)* nu * np.sqrt(V_tk[i-1])* z[i] \
                      + 1/gamma(1.- alpha) * wl*np.exp(-xl*tau)@J_tn[i-1,:]      ),0) 
            
        return V_tk
    
    
    def volMF(self, texp, n_exp=10, V0=0, alpha=0.0, kappa=0.3, theta=0.5, nu=0.01, time_steps=1_000):    
        '''
        '''
        tau = texp/time_steps
        eta = np.linspace(0,n_exp,n_exp+1)*n_exp**(-0.2)/texp*(np.sqrt(10)*alpha/(2+alpha))**0.4
        gammaj = np.zeros(n_exp+1) 
        c = np.zeros(n_exp+1)
        
        f = lambda x: x**(alpha-1)/(gamma(1-alpha)*gamma(alpha))
        g = lambda x: x**alpha/(gamma(1-alpha)*gamma(alpha))

        
        for j in range(1,n_exp+1):
            c[j] = quad(f,eta[j-1],eta[j])[0]
            gammaj[j] = 1/c[j]*quad(g,eta[j-1],eta[j])[0]
        
        c = np.delete(c,0,0)
        gammaj = np.delete(gammaj,0,0)
        eta = np.delete(eta,0,0)
        
        z = np.random.normal(size=(time_steps+1)) 
        V_tk = V0 * np.ones(time_steps+1) 
        V_tk_j = np.zeros([time_steps+1,n_exp])
        tk=np.linspace(0, texp, num=time_steps+1)  

        for i in range(1,self.time_steps+1):
                        
            V_tk_j[i,:] = (V_tk_j[i-1,:] - kappa *tau * V_tk[i-1] + nu * np.sqrt(V_tk[i-1] * tau) * z[i]) / (1 + tau * gammaj )
            
            V_tk[i] = np.max((V_tk[0] + kappa * theta * c/gammaj @ (1-np.exp(-i*tau*gammaj)) + c @  V_tk_j[i,:]  ),0) 
        
        return V_tk
        
        
        
        
    #def price
        

In [162]:
rh=RoughHeston(texp=1,time_steps=100,V0=0.2,alpha=0.4,kappa=0.3,theta=0.5,nu=0.01)

In [163]:
rh.volMEM(texp=1,time_steps=100,V0=0.2,alpha=0.4,kappa=0.3,theta=0.5,nu=0.01)

array([0.2       , 0.20008663, 0.21105227, 0.20876175, 0.21592688,
       0.2180347 , 0.21766623, 0.22108224, 0.21780716, 0.22064256,
       0.22590975, 0.22917418, 0.22270104, 0.22087306, 0.22894953,
       0.22738635, 0.23013384, 0.23425787, 0.23479083, 0.23101305,
       0.23495969, 0.23839941, 0.23757857, 0.2375767 , 0.24325695,
       0.24145548, 0.24902375, 0.24844618, 0.25773014, 0.25179485,
       0.25622938, 0.25638491, 0.25578863, 0.26235886, 0.26353914,
       0.26098662, 0.25673538, 0.25061565, 0.26002699, 0.26407204,
       0.25975531, 0.26371174, 0.25369886, 0.25963913, 0.25112081,
       0.252756  , 0.25428774, 0.25167817, 0.26222282, 0.25906988,
       0.25811496, 0.25142332, 0.25610739, 0.24710986, 0.26272284,
       0.26173897, 0.24966342, 0.25686365, 0.25579206, 0.26019535,
       0.25509617, 0.25817004, 0.263428  , 0.26856378, 0.27523334,
       0.26657576, 0.26704108, 0.26691178, 0.26671267, 0.27927225,
       0.28056346, 0.2697795 , 0.27709275, 0.28120017, 0.27809

In [164]:
rh.volFast(texp=1,n_exp=10,time_steps=100,V0=0.2,alpha=0.4,kappa=0.3,theta=0.5,nu=0.01)

array([0.2       , 0.20704048, 0.20676023, 0.20627471, 0.20876606,
       0.20746195, 0.20525634, 0.20864659, 0.20636027, 0.20703737,
       0.20811705, 0.20876532, 0.20984808, 0.2088558 , 0.20433488,
       0.20641752, 0.20864859, 0.20730983, 0.208062  , 0.20814148,
       0.21082736, 0.21439812, 0.20968873, 0.20768687, 0.21154958,
       0.21145047, 0.21141524, 0.2071582 , 0.21011203, 0.21130332,
       0.21214853, 0.20921881, 0.21166455, 0.20881715, 0.21126709,
       0.21077562, 0.21177678, 0.20996546, 0.21454722, 0.21387003,
       0.20923123, 0.20951634, 0.20791912, 0.21247396, 0.21151849,
       0.21250629, 0.21253338, 0.2133028 , 0.21244074, 0.20889328,
       0.21200035, 0.21727688, 0.21470436, 0.21312262, 0.21072899,
       0.21503249, 0.21121265, 0.211476  , 0.21201497, 0.2142161 ,
       0.21277531, 0.20917504, 0.21297411, 0.21338939, 0.21479474,
       0.2135522 , 0.21495716, 0.21159094, 0.21573938, 0.21304921,
       0.21540516, 0.21740732, 0.21445171, 0.21552399, 0.21625

In [165]:
rh.volMF(texp=1,n_exp=10,time_steps=100,V0=0.2,alpha=0.4,kappa=0.3,theta=0.5,nu=0.01)

array([0.2       , 0.20058899, 0.20259674, 0.20390938, 0.20419564,
       0.20590211, 0.2077034 , 0.20799022, 0.21004791, 0.20986017,
       0.21099792, 0.21212467, 0.21265944, 0.21325054, 0.21390629,
       0.21478271, 0.21659633, 0.21625662, 0.21773867, 0.21891108,
       0.21956233, 0.21997956, 0.22170176, 0.22228574, 0.22336399,
       0.22433454, 0.22481104, 0.22589731, 0.22633485, 0.22726992,
       0.22892171, 0.22929086, 0.23028981, 0.23171092, 0.23332503,
       0.2348185 , 0.23573049, 0.23721659, 0.23784726, 0.23855499,
       0.23972583, 0.24067026, 0.24064637, 0.24200026, 0.2441548 ,
       0.24465721, 0.2470464 , 0.24868495, 0.24840796, 0.24943792,
       0.24868336, 0.24873184, 0.24786663, 0.24711036, 0.24765605,
       0.24924372, 0.24926373, 0.24921506, 0.25075559, 0.25141944,
       0.25133213, 0.25214399, 0.25297739, 0.25374445, 0.25406974,
       0.25593625, 0.2566929 , 0.25657139, 0.25750325, 0.25830609,
       0.25864692, 0.25945018, 0.26132845, 0.26256591, 0.26295

In [ ]:
#Figure

In [ ]:
#Table